In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from loguru import logger

In [4]:
import os
import sys
sys.path.append('..')

In [5]:
from pals.pimp_tools import get_pimp_API_token_from_env, PIMP_HOST, download_from_pimp
from pals.evaluation import run_experiment, _select_significant_entries, _compute_prec_rec_f1
from pals.common import *
from pals.feature_extraction import DataSource
from pals.pathway_analysis import PALS

2019-12-29 17:09:39.301 | WARNING  | pals.reactome:get_neo4j_driver:14 - Using a default neo4j server: bolt://localhost:7687
2019-12-29 17:09:39.304 | WARNING  | pals.reactome:get_neo4j_driver:19 - Using a default neo4j username or password: neo4j
2019-12-29 17:09:40.386 | INFO     | pals.reactome:get_neo4j_driver:24 - Created graph database driver for bolt://localhost:7687 (neo4j)


# HAT Data Analysis

This notebook is used to generate resampled data and run the different methods for comparison in the manuscript. Results are analysed in `PALS_HAT_experiment_evaluation.ipynb`

### Load data

Assume token is stored in environmental variable *PIMP_API_TOKEN*

In [6]:
token = get_pimp_API_token_from_env()

In [7]:
analysis_id_plasma = 636
int_df_plasma, annotation_df_plasma, experimental_design_plasma = download_from_pimp(token, PIMP_HOST, analysis_id_plasma, 'kegg')

2019-12-29 17:09:40.558 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\joewa\AppData\Local\Temp\pimp_analysis_636.p


In [8]:
analysis_id_csf = 635
int_df_csf, annotation_df_csf, experimental_design_csf = download_from_pimp(token, PIMP_HOST, analysis_id_csf, 'kegg')

2019-12-29 17:09:40.665 | DEBUG    | pals.pimp_tools:download_from_pimp:119 - Trying to load data from temp file: C:\Users\joewa\AppData\Local\Temp\pimp_analysis_635.p


### Create Data Sources

In [9]:
database_name = DATABASE_PIMP_KEGG
ds_plasma = DataSource(int_df_plasma, annotation_df_plasma, experimental_design_plasma, database_name)
ds_csf = DataSource(int_df_csf, annotation_df_csf, experimental_design_csf, database_name)

2019-12-29 17:09:40.752 | DEBUG    | pals.feature_extraction:__init__:38 - Using PiMP_KEGG as database
2019-12-29 17:09:40.752 | DEBUG    | pals.feature_extraction:get_database:101 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2019-12-29 17:09:40.774 | DEBUG    | pals.feature_extraction:__init__:51 - Mapping pathway to unique ids
2019-12-29 17:09:40.779 | DEBUG    | pals.feature_extraction:__init__:65 - Creating dataset to pathway mapping
2019-12-29 17:09:41.144 | DEBUG    | pals.feature_extraction:__init__:93 - Computing unique id counts
2019-12-29 17:09:41.153 | DEBUG    | pals.feature_extraction:__init__:38 - Using PiMP_KEGG as database
2019-12-29 17:09:41.153 | DEBUG    | pals.feature_extraction:get_database:101 - Loading C:\Users\joewa\Work\git\PALS\pals\data\PiMP_KEGG.json.zip
2019-12-29 17:09:41.176 | DEBUG    | pals.feature_extraction:__init__:51 - Mapping pathway to unique ids
2019-12-29 17:09:41.180 | DEBUG    | pals.feature_extraction:__init__:65 - Crea

In [10]:
ds_plasma.get_measurements().shape

(15584, 60)

In [11]:
ds_csf.get_measurements().shape

(8154, 57)

# Run PALS Experiments

In [12]:
plage_weight = 5
hg_weight = 1
proportions = np.array([0.25, 0.50, 0.75])
n_iter = 500
results = {}

#### Disable debug logging

In [13]:
set_log_level_info()

#### Run on Plasma samples

In [14]:
data_source = ds_plasma
experiment_name = 'plasma'
case = 'Stage1'
control = 'Control'
significant_column = 'PiMP_KEGG Stage1/Control comb_p'

In [15]:
res = run_experiment(experiment_name, data_source, case, control, proportions, significant_column, n_iter, plage_weight, hg_weight)
results[experiment_name] = res

2019-12-29 17:09:41.896 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=0 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:09:51.450 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=1 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:09:59.608 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=2 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:10:08.671 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=3 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:10:17.100 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=4 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:10:26.428 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=3896 iter=5 PALS experiment=plasma case=Stage1 control=Control
2019-12-29 17:10:35.327 | INFO     | pals.eval

#### Run on CSF samples

In [16]:
data_source = ds_csf
experiment_name = 'csf'
case = 'Stage_2'
control = 'Control'
significant_column = 'PiMP_KEGG Stage_2/Control comb_p'

In [17]:
res = run_experiment(experiment_name, data_source, case, control, proportions, significant_column, n_iter, plage_weight, hg_weight)
results[experiment_name] = res

2019-12-29 22:36:16.789 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=0 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:36:24.570 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=1 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:36:32.789 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=2 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:36:40.764 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=3 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:36:48.678 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=4 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:36:56.451 | INFO     | pals.evaluation:run_experiment:36 - prop=0.250000 n_sample=2038 iter=5 PALS experiment=csf case=Stage_2 control=Control
2019-12-29 22:37:04.240 | INFO     | pals.evaluation:run_e

#### Save Results

In [18]:
save_obj(results, os.path.join('test_data', 'HAT', 'HAT_results.p'))